In [1]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from numba import jit
from sklearn.linear_model import LinearRegression  # for linear regression
from sklearn.tree import DecisionTreeClassifier  # for decision tree mining
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

In [2]:

pip install tensorflow

  Using cached tensorflow-2.8.0-cp37-cp37m-win_amd64.whl (437.9 MB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-13.0.0-py2.py3-none-win_amd64.whl (13.9 MB)
  Using cached numpy-1.21.5-cp37-cp37m-win_amd64.whl (14.0 MB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
Processing c:\users\20201158\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2\termcolor-1.1.0-py3-none-any.whl
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.44.0-cp37-cp37m-win_amd64.whl (3.4 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached Markdown-3.3.6-py3-none-any.whl (97 kB)
  Using cach

ERROR: pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
ERROR: pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\numpy\\compat\\py3k.py'
Consider using the `--user` option or check the permissions.



In [ ]:
file_export = 'export2012.csv'
data = pd.read_csv(file_export)
df_sorted= data.sort_values(by=['case', 'startTime'])
data = df_sorted.copy().loc[:,['case', 'event', 'startTime', 'completeTime']]

In [ ]:
data = pd.read_csv(file_export)

In [ ]:
data_sorted = data.sort_values(by=['case', 'event','completeTime'])

In [ ]:
data_iterrated = data.copy()

@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]
        idx+=1
    return res

data_iterrated["next_task"] = calculator_nb(data_iterrated['case'].values, data_iterrated["event"].values)

In [ ]:
data_train, data_test = train_test_split(data_iterrated,test_size=0.2, shuffle=False)

In [ ]:
data_iterrated.head()

In [ ]:
list_of_events = data_iterrated["event"].unique()
event_to_num = {}
i=0
for event in list_of_events:
    event_to_num[str(event)] = i
    i += 1

In [ ]:
event_to_num['None'] = 23

Naive predictor. First, event predictor is constructed.

In [ ]:
#For each event, counts how many times another event occurs after it.
#Indexed based on list_of_events
count = {}
# initialise the count dictionary with zeros, otherwise it does not work
for event in list_of_events:
        count[str(event)] = [0 for i in range(len(list_of_events))]
 

for event in list_of_events:
    for k in range(len(list_of_events)):
        count[str(event)][k] = data_train[ (data_train['event'] == event) & (data_train['next_task'] == list_of_events[k]) ].count()['event']
            
count

In [ ]:
#The main part of the naive predictor
#Takes the above dictionary and finds the most frequenly occuring consecutive
#event after each event
#Basically, it is our trained model that can be mapped to the original dataset
count2 = count
index = {}
for event in list_of_events:
    max = 0
    for k in range( len(list_of_events)-1 ):
        if count2[event][k] > max:
            max = count2[event][k]
            #print(max)
            index2 = k
    index[event] = index2
index

In [ ]:
#Turns the above into an event to event dictionary 

prediction = {}
for event in list_of_events:
    prediction[event] = list_of_events[index[event]]
    
prediction

In [ ]:
#Adds predicted event column
df_predicted_event = data_test.copy()
df_predicted_event["predicted_event"] = ["" for i in df_predicted_event["event"]]
df_predicted_event['predicted_event'] = df_predicted_event["event"].map(prediction)
df_predicted_event.head(10)

In [ ]:
df_new3 = df_predicted_event.copy()
df_new3["next_task"] = df_new3["next_task"].map(event_to_num)
df_new3["predicted_event"] = df_new3["predicted_event"].map(event_to_num)
df_new3.head()

In [ ]:
y_true = df_new3["next_task"].to_numpy().astype(int)
y_pred = df_new3["predicted_event"].to_numpy().astype(int)
y_true, y_pred

In [ ]:
f1_score(y_true, y_pred, average='micro')

Naive time predictor

In [ ]:
df_predicted_time = data_train.copy()
df_predicted_time["duration"] = pd.to_datetime(df_predicted_time["completeTime"]) - pd.to_datetime(df_predicted_time["startTime"])

#Sums up count for each event and the time each event takes
events_count = df_predicted_time.groupby("event")['duration'].agg('count')
event_duration_sum = df_predicted_time.groupby("event")['duration'].agg('sum')

#Computes average duration per event (basically our trained data that can be mapped onto test data)
duration_per_event = event_duration_sum / events_count 

In [ ]:
#Final cleanup
df_predicted = df_predicted_event.copy()
df_predicted["predicted_duration"] = df_predicted['event'].map(duration_per_event)
df_predicted = df_predicted.drop(['next_task'], axis=1)
df_predicted.head(10)

In [ ]:
#Adding predicted start time of next event
df_predicted_final = df_predicted.copy()
df_predicted_final['predicted_time'] = df_predicted_final["predicted_duration"] + pd.to_datetime(df_predicted_final["startTime"])
df_predicted_final.head(10)

Linear regression

In [ ]:
data_iterrated_two = data_iterrated["event"].copy()
data_iterrated_two

In [ ]:
for i in range(len(data_iterrated_two)):
    data_iterrated_two[i] = event_to_num[data_iterrated_two[i]]

In [ ]:
event_to_num

In [ ]:
next_task = data_iterrated["next_task"].copy()
for i in range(len(next_task)):
    try:
        next_task[i] = event_to_num[next_task[i]]
    except:
        next_task[i] = 23

In [ ]:
data_to_lr = data_iterrated.copy()
data_to_lr["event"] = data_iterrated_two
data_to_lr["next_task"] = next_task
data_to_lr

In [ ]:
x = data_to_lr[["event"]]
y = data_to_lr["next_task"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
naive_predictor = LinearRegression()
naive_predictor.fit(X_train, y_train)
naive_predictor.score(X_test, y_test)
prediction = naive_predictor.predict(data_to_lr[["event"]]).round()

In [ ]:
result_data = data_to_lr.copy()
result_data["prediction"] = prediction
result_data["prediction"] = result_data["prediction"].astype(int)

In [ ]:
result_data

In [ ]:
num_to_event = {v: k for k, v in event_to_num.items()}
num_to_event

In [ ]:
predicted_event_str = result_data["prediction"].copy()
for i in range(len(predicted_event_str)):
    predicted_event_str[i] = num_to_event[predicted_event_str[i]]

In [ ]:
predicted_event_str

In [ ]:
result_data_str = data_iterrated.copy()
result_data_str["prediction"] = predicted_event_str

In [ ]:
result_data_str["duration"] = pd.to_datetime(result_data_str["completeTime"]) - pd.to_datetime(result_data_str["startTime"])
result_data_str.head()

In [ ]:

events_count = result_data_str.groupby("event")['duration'].agg('count')
event_duration_sum = result_data_str.groupby("event")['duration'].agg('sum')

duration_per_event = event_duration_sum / events_count

@jit(parallel = True)
def make_prediction(event, dict):
    res = np.empty(len(event), dtype=object)
    for idx,_ in enumerate(event):    
        res[idx] = dict[event[idx]]
    return res

result_data_str["duration_prediction"] = make_prediction(result_data_str['event'].values, duration_per_event)

In [ ]:
result_data_str.head(5)

More advanced algorithms (from Hristo)

In [ ]:
df_filtered = result_data[result_data["case"] < len(result_data)]
df_prediction = result_data[result_data["case"] >= len(result_data)][:100]

df_filtered=df_filtered.dropna() #Check later if dropna messes with the dataset through "None" tasks being interpreted as NaN
df_prediction=df_prediction.dropna()
df_filtered["startTime"] = pd.to_datetime(df_filtered["startTime"])
df_prediction["startTime"] = pd.to_datetime(df_prediction["startTime"])

In [ ]:
#OLS regression
import statsmodels.api as sm
df_filtered = result_data[result_data["case"] < len(result_data)]
df_prediction = result_data[result_data["case"] >= len(result_data)]


df_filtered=df_filtered.dropna()
df_prediction=df_prediction.dropna()
df_filtered["startTime"] = pd.to_datetime(df_filtered["startTime"])
df_prediction["startTime"] = pd.to_datetime(df_prediction["startTime"])

y = np.array(pd.to_numeric(df_filtered["startTime"].values), dtype=float)
X = np.array(df_filtered["event"].values, dtype=float)
olsmod = sm.OLS(y, X)
olsres = olsmod.fit()
ypred = olsres.predict(X)
Xnew = np.array(df_prediction["event"].values, dtype=float)
ynewpred = olsres.predict(Xnew) 
df_prediction["time_OLS"] = pd.to_datetime(ynewpred)

In [ ]:
# Lasso Regression
from sklearn import linear_model

y = np.floor(pd.to_numeric(df_filtered["startTime"].values), dtype=float)
X = df_filtered[["event", "case"]]

lassoReg = linear_model.Lasso()
lassoReg.fit(X, y)

Xnew = np.array(df_prediction[["event", "case"]].values, dtype=float)
df_prediction["time_Lasso"] = pd.to_datetime(lassoReg.predict(Xnew))

Random Fores and Decision Tree for event

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

In [ ]:
# Prep data
result_data.sort_values(by=["case", "event"], inplace=True)
result_data_copy = result_data.copy()
result_data_copy['startTime'] = pd.to_numeric(pd.to_datetime(result_data_copy['startTime']))
result_data_copy['completeTime'] = pd.to_numeric(pd.to_datetime(result_data_copy['completeTime']))
result_data_copy['REG_DATE'] = pd.to_numeric(pd.to_datetime(result_data_copy['REG_DATE']))
result_data_copy.drop(columns='prediction', inplace=True)

In [ ]:
# Split training and test and ensure floating point numbers
X = result_data_copy.drop(['next_task'], axis=1)
y = result_data_copy['next_task']




X_train, X_test, y_train, y_test = train_test_split(X, y)
event_columns = X_train.columns
X_train = X_train.values.astype("float32")
y_train = y_train.values.astype("float32")
X_test = X_test.values.astype("float32")
y_test = y_test.values.astype("float32")

In [ ]:
event_columns

In [ ]:
# Fitting and prediction 
DT = DecisionTreeClassifier()
RF = RandomForestClassifier()


DT_fit = DT.fit(X_train, Y_train)
RF_fit = RF.fit(X_train, Y_train)

DT_pred = DT_fit.predict(X_test)
RF_pred = RF_fit.predict(X_test)

In [ ]:
DT_model = SelectFromModel(DT_fit, prefit=True)
X_new = DT_model.transform(X)
#X_new.shape


In [ ]:
X_new_df = pd.DataFrame(X_new, columns= ["event", "startTime", "completeTime"])
X_new_df

In [ ]:
# Accuracy
DT_pred = np.round(DT_pred)
RF_pred = np.round(RF_pred)
print("Decision Trees is %f percent accurate" % (accuracy_score(DT_pred, Y_test)*100))
print("Random Forests is %f percent accurate" % (accuracy_score(RF_pred, Y_test)*100))

In [ ]:
# Add to df
test_df["event_DT"] = DT_pred
test_df["event_RF"] = RF_pred

In [ ]:
y = y

X_n_train, X_n_test, y_n_train, y_n_test = train_test_split(X_new_df, y)
event_columns = X_n_train.columns
X_n_train = X_n_train.values.astype("float32")
y_n_train = y_n_train.values.astype("float32")
X_n_test = X_n_test.values.astype("float32")
y_n_test = y_n_test.values.astype("float32")

In [ ]:
DT_n_fit = DT.fit(X_n_train, y_n_train)
DT_n_pred = DT_fit.predict(X_n_test)
DT_n_pred = np.round(DT_n_pred)
print("Decision Trees with feature selection is %f percent accurate" % (accuracy_score(DT_n_pred, Y_test)*100))

LSTM for time

In [ ]:
df_result = result_data.sort_values(by=["case", "event"])

df_result["duration"] = pd.to_numeric(pd.to_datetime(df_result["completeTime"]) - pd.to_datetime(df_result["startTime"]))
df_result['REG_DATE'] = pd.to_numeric(pd.to_datetime(df_result['REG_DATE']))
df_result['completeTime'] = pd.to_numeric(pd.to_datetime(df_result['completeTime']))
df_result['startTime'] = pd.to_numeric(pd.to_datetime(df_result['startTime']))

df_filtered = df_result[df_result["case"] < len(df_result)]
df_prediction = df_result[df_result["case"] >= len(df_result)]



df_filtered_new = [v for _, v in df_filtered.groupby('event')]
dictionary_event_startTime = np.empty(len(df_filtered_new), dtype=object) 
for idx, val in enumerate(df_filtered_new):
    dictionary_event_startTime[idx] = val["startTime"]

listVal = df_filtered
listVal

columnNames = ['case', 'event', 'startTime', 'completeTime', 'AMOUNT_REQ', 'REG_DATE']

listValSelected = listVal[columnNames]
listValSelected_prediction = df_prediction[columnNames]
listValSelected_prediction = listValSelected_prediction.values.astype('float32')
listValDuration_prediction = df_prediction['duration']
listValDuration_prediction = listValDuration_prediction.values.astype('float32')
listValDuration = listVal['duration'].astype('float32')

listValSelected = listValSelected.values.astype('float32')

In [ ]:
# choose a number of time steps
n_steps = len(listValSelected[0])
# split into samples


n_features = 1
X = listValSelected.reshape((listValSelected.shape[0], listValSelected.shape[1], n_features))
y = listValDuration

# define model
model = Sequential()
model.add(LSTM(50, input_shape=(n_steps, n_features), return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(50, activation='relu'))
# Dropout for regularization
# model.add(Dropout(0.5))
# model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=10, verbose=2, workers=-1)

# demonstrate prediction
x_input = listValSelected_prediction

x_input = x_input.reshape((x_input.shape[0], n_steps, n_features))

In [ ]:
yhat = model.predict(x_input, verbose=2)
lenVal_len = len(listValDuration_prediction) 

print(mean_absolute_error(listValDuration_prediction, yhat.flatten()[:lenVal_len])) # 286867030000.0 nanoseconds which is 5 minutes
test_df["duration_prediction"] = yhat.flatten()[:len(test_df)]

Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest

select = SelectKBest(k=3) # takes best 3 arguments 
z = select.fit_transform(listValSelected, listValDuration)
filter = select.get_support()
# 3 best features['case' 'event' 'AMOUNT_REQ'] for time
print(np.extract(filter, columnNames))


select = SelectKBest(k=3) # takes best 3 arguments for event
z = select.fit_transform(X_train, Y_train)
filter = select.get_support()
print(np.extract(filter, event_columns)) # ['event' 'AMOUNT_REQ' 'org:resource']

Neural network for event prediction

In [ ]:
import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom
from numba import jit
import matplotlib
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from tqdm.keras import TqdmCallback
import seaborn as sns
from sklearn.linear_model import HuberRegressor
import statsmodels.api as sm
from sklearn.linear_model import RANSACRegressor

In [ ]:
df = pd.read_csv('export2018.csv')

In [ ]:
df = df.sort_values(by=['case','startTime'])
df = df.reset_index()
#A function for determining the true next event for each event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]

        idx+=1
    return res

df['next_event'] = calculator_nb(df['case'].values, df['event'].values)


#A function for determining the true next event for each event
@jit(parallel = True)
def calculator_nb(case, startTime):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = startTime[idx + 1]
        else:
            res[idx] = startTime[idx]

        idx+=1
    return res

df['completeTime'] = calculator_nb(df['case'].values, df['startTime'].values)
df.at[2514265, 'completeTime'] = df.at[2514265, 'startTime']

df['startTime'] =  pd.to_datetime(df['startTime'])
df['completeTime'] =  pd.to_datetime(df['completeTime'])
df['duration'] = df['completeTime'] - df['startTime']

duration = df['duration']
duration = duration / np.timedelta64(1, 's')
df['duration'] = duration

In [ ]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [ ]:
train = train.reset_index()
validate = validate.reset_index()
test = test.reset_index()

In [ ]:
ordinal_encoder = OrdinalEncoder()
label_encoder = LabelEncoder()

In [ ]:
def prepfeatures_OLS(df_name):
    startTime = pd.to_datetime(df_name['startTime'])
    
    for i in range(len(startTime)):
        startTime[i] = startTime[i].timestamp()
        
    startTime = np.array(startTime).reshape(-1,1)
    
    event = df_name['event'].to_numpy()
    event = event.reshape(-1,1)
    event = ordinal_encoder.fit_transform(event)
    
    #payment_actual0 = normalize(df_name,'payment_actual0')
    #penalty_amount0 = normalize(df_name,'penalty_amount0')
    #number_parcels = normalize(df_name,'number_parcels')
    #area = normalize(df_name,'area')
    payment_actual0 = df_name['payment_actual0'].to_numpy()
    penalty_amount0 = df_name['penalty_amount0'].to_numpy()
    number_parcels = df_name['number_parcels'].to_numpy()
    area = df_name['area'].to_numpy()
    
    X = []
    for i in range(len(event)):
        current = startTime[i]
        current = np.append(current, event[i])
        current = np.append(current, payment_actual0[i])
        current = np.append(current, penalty_amount0[i])
        current = np.append(current, number_parcels[i])
        current = np.append(current, area[i])
        X.append(current)
        
    return np.array(X, dtype=float)

In [ ]:
def preplabels_OLS(df_name):
    duration = df_name['duration'].to_numpy()
    return np.array(duration, dtype=float)

In [ ]:
X = prepfeatures_OLS(train)
y = preplabels_OLS(train)
huber = HuberRegressor().fit(X, y)
X_test = prepfeatures_OLS(test)
test['predicted'] = huber.predict(X_test)
test['error'] = np.absolute(test['duration'] - test['predicted'])
test['error'].mean()

In [ ]:
def normalize(df_name, col_name):
    col_as_array = df_name[col_name].to_numpy()
    col_as_array = np.where(col_as_array == 0, 0.5, col_as_array)
    col_as_array_norm = np.log10(col_as_array)
    mean = col_as_array_norm.mean()
    stdev = col_as_array_norm.std()
    epsilon = 0.01
    return (col_as_array_norm - mean) / (stdev + epsilon)

def prepfeatures(df_name):
    event = df_name['event'].to_numpy()
    event = event.reshape(-1,1)
    event = ordinal_encoder.fit_transform(event)
    
    number_parcels = normalize(df_name,'number_parcels')
    payment_actual0 = normalize(df_name,'payment_actual0')
    area = normalize(df_name,'area')
    cross_compliance = normalize(df_name,'cross_compliance')
    penalty_amount0 = normalize(df_name,'penalty_amount0')
    
    features = []
    for i in range(len(event)):
        current = event[i]
        current = np.append(current,number_parcels[i])
        current = np.append(current,payment_actual0[i])
        current = np.append(current,area[i])
        current = np.append(current,cross_compliance[i])
        current = np.append(current,penalty_amount0[i])
        features.append(current)
        
    return np.array(features)

def preplabels(df_name):
    labels = df_name['next_event'].to_numpy()
    labels = label_encoder.fit_transform(labels)
    labels = labels.reshape(-1, 1)
    
    return np.array(labels)

features = prepfeatures(train)
labels = preplabels(train)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(35, activation='relu'),
    keras.layers.Dense(42, activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(features,labels,epochs=5,verbose=1, n_jobs=-1)

In [ ]:
features_test = prepfeatures(test)
labels_test = preplabels(test)

In [ ]:
score = model.evaluate(features_test, labels_test, verbose=1)
print(score)

In [ ]:
#Error model